# 画像を使ったプロンプト


## Vision（画像理解）機能

Claude 3 ファミリーのモデルには、画像を理解・分析できる Vision 機能があります。これにより、会話に **テキストと画像の両方** を入力として与えられ、より強力なユースケースが実現できます。Opus / Sonnet / Haiku はいずれも画像を扱えますが、Claude 3.5 Sonnet が最も Vision が強いため、このレッスンでは主にそれを使います。


Claude に画像を渡すときも、テキストだけの会話で見てきたのと同じ `messages` 形式を使います。例えばテキストのみの user メッセージは次のようになります：

```python
messages = [
    {
        "role": "user",
        "content": "tell me a joke"
    }
]
```


In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [
    {"role": "user", "content": "tell me a joke"}
]

response = client.messages.create(
    messages=messages,
    model="claude-3-haiku-20240307",
    max_tokens=200
)
print(response.content[0].text)

ここまで見ていませんでしたが、メッセージの `content` は **リスト（content blocks のリスト）** にすることもできます。例えば次の代わりに：

```python
messages = [
    {"role": "user", "content": "tell me a joke"}
]
```

次のように書けます：

```python
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "tell me a joke"},
        ]
    }
]
```

次の 2 つは同じ意味です：

```python
{"role": "user", "content": "Tell me a story"}
```

```python
{"role": "user", "content": [{"type": "text", "text": "Tell me a story"}]}
```

実際に試してみましょう：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "tell me a joke"},
        ]
    }
]

response = client.messages.create(
    messages=messages,
    model="claude-3-haiku-20240307",
    max_tokens=200
)
print(response.content[0].text)

この通り、動きます！リストには必要なだけ content block を追加できます。次の例では複数のテキストブロックを入れています：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "who"},
            {"type": "text", "text": "made"},
            {"type": "text", "text": "you?"},
        ]
    }
]

response = client.messages.create(
    messages=messages,
    model="claude-3-haiku-20240307",
    max_tokens=200
)
print(response.content[0].text)

なぜわざわざこうするのでしょうか？テキストだけのプロンプトでは、普通はあまり必要ありません。しかし **マルチモーダル（画像＋テキスト）** のプロンプトでは、この形式が必要になります。


Claude に画像を渡すには、画像用の content block を書きます。例：

```python
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": "/9j/4AAQSkZJRg..."
                }
            }
        ]
    }
]
```

この図は、画像を渡す際に必要な情報を示しています：

![image_message_format](images/image_message_format.png)

この画像ブロックは、次のプロパティを持ちます：

* `type` - 画像のエンコード形式（現時点では `base64`）
* `media_type` - 画像のメディアタイプ（`image/jpeg`, `image/png`, `image/gif`, `image/webp` をサポート）
* `data` - 画像データ本体（base64 文字列）


## 画像だけでプロンプトする

多くの場合は画像に加えてテキストも添えますが、画像だけを渡すことも問題ありません。試してみましょう！このレッスン用に `prompting_images` フォルダにいくつか画像を入れてあります。まずは Python で画像を表示してみます：


In [ ]:
from IPython.display import Image
Image(filename='./prompting_images/uh_oh.png') 

Wikimedia Commons, CC-BY-SA


では、この画像を Claude に渡してみます。最初のステップは、モデルに送る base64 エンコード済みの画像データ文字列を作ることです。コードは少し複雑に見えますが、やっていることは次の通りです：

1. ファイルをバイナリ読み込み（read binary）で開く
2. ファイル全体を bytes として読む
3. bytes を base64 エンコードする
4. base64 の bytes を文字列にする


In [ ]:
import base64

# opens the image file in "read binary" mode
with open("./prompting_images/uh_oh.png", "rb") as image_file:

    #reads the contents of the image as a bytes object
    binary_data = image_file.read() 

    #encodes the binary data using Base64 encoding
    base_64_encoded_data = base64.b64encode(binary_data) 

    #decodes base_64_encoded_data from bytes to a string
    base64_string = base_64_encoded_data.decode('utf-8')


生成された `base64_string` を見ても、人間にはほとんど意味が分かりません。先頭 100 文字だけ見てみましょう：


In [ ]:
base64_string[:100]

画像データが文字列になったので、次は Claude に送る `messages` リストを正しい形にします：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [{
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_string
            },
        }]
    }
]

最後に、この `messages` を Claude に送って、どんな応答が返るか見てみましょう！


In [ ]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

今回は特に指示を与えていないため、Claude は画像の説明を始めます。


## 画像＋テキストのプロンプト

次は画像とテキストを両方含むプロンプトを送ってみましょう。やることは簡単で、user メッセージの `content` に **2つ目のブロック**としてテキストブロックを追加するだけです。


In [ ]:
messages = [
    {
        "role": "user",
        "content": [{
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_string
            },
        },
        {
            "type": "text",
            "text": "What could this person have done to prevent this?"
        }]
    }
]

画像ブロックとテキストブロックを強調した図はこちらです：

![image_and_text_prompt](images/image_and_text_prompt.png)


Claude にリクエストを送り、どう返ってくるか見てみましょう：


In [ ]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

## 複数画像

user メッセージの `content` に複数の画像ブロックを入れることで、Claude に複数画像を渡せます。例：

```python
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": image1_media_type,
                    "data": image1_data,
                },
            },
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": image2_media_type,
                    "data": image2_data,
                },
            },
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": image3_media_type,
                    "data": image3_data,
                },
            },
            {"type": "text", "text": "How are these images different?"},
        ],
    }
]
```


### 画像ブロック用ヘルパーを作る

画像を扱う場面、特に動的なスクリプトでは、毎回手で画像ブロックを書くのは面倒になりがちです。そこで、適切な形式の画像ブロックを生成する簡単なヘルパー関数を書いてみましょう。


In [ ]:
import base64
import mimetypes

def create_image_message(image_path):
    # Open the image file in "read binary" mode
    with open(image_path, "rb") as image_file:
        # Read the contents of the image as a bytes object
        binary_data = image_file.read()
    
    # Encode the binary data using Base64 encoding
    base64_encoded_data = base64.b64encode(binary_data)
    
    # Decode base64_encoded_data from bytes to a string
    base64_string = base64_encoded_data.decode('utf-8')
    
    # Get the MIME type of the image based on its file extension
    mime_type, _ = mimetypes.guess_type(image_path)
    
    # Create the image block
    image_block = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": mime_type,
            "data": base64_string
        }
    }
    
    
    return image_block

この関数は画像パスを受け取り、Claude に送る message にそのまま入れられる辞書を返します。拡張子から mime type を推定する処理も含まれています。

別の画像で試してみましょう：


In [ ]:
Image("./prompting_images/animal1.png")

新しいヘルパー関数を使って、Claude にリクエストを送ってみます：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./prompting_images/animal1.png")
        ]
    }
]

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

次は、プロンプトに画像とテキストを組み合わせる例です：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./prompting_images/animal1.png"),
            {"type": "text", "text": "Where might I find this animal in the world?"}
        ]
    }
]

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

では、Claude に複数画像を渡してみましょう。動物の画像が 3 枚あります：


In [ ]:
from IPython.display import display
display(Image("./prompting_images/animal1.png", width=300))

In [ ]:
display(Image("./prompting_images/animal2.png", width=300))

In [ ]:
display(Image("./prompting_images/animal3.png", width=300))

3 枚すべてを 1 つのメッセージで Claude に渡し、テキストで「What are these animals?」と質問してみましょう：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message('./prompting_images/animal1.png'),
            create_image_message('./prompting_images/animal2.png'),
            create_image_message('./prompting_images/animal3.png'),
            {"type": "text", "text": "what are these animals?"}
        ]
    }
]

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

これはうまく動きます！ただし、Claude 3 Haiku のような能力が低めのモデルで同じことをすると、結果が悪くなる場合があります：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message('./prompting_images/animal1.png'),
            create_image_message('./prompting_images/animal2.png'),
            create_image_message('./prompting_images/animal3.png'),
            {"type": "text", "text": "what are these animals?"}
        ]
    }
]

response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

この応答は物足りません。整理すると、私たちは「ハクトウワシ」「水中のグリズリーベア」「ヤマアラシのクローズアップ」の 3 枚を渡し、続けて "What are these animals?" と質問しました。しかし Claude はヤマアラシの説明だけを返しました。

Claude 3.5 Sonnet では問題になりにくいのですが、他のモデルでは、各画像にテキストブロックでラベルを付けると改善することがあります。例えば "Image 1", "Image 2" のように簡単にラベルするだけでも大きく変わります。

試してみましょう：


In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Image 1:"},
            create_image_message('./prompting_images/animal1.png'),
            {"type": "text", "text": "Image 2:"},
            create_image_message('./prompting_images/animal2.png'),
            {"type": "text", "text": "Image 3:"},
            create_image_message('./prompting_images/animal3.png'),
            {"type": "text", "text": "what are these animals?"}
        ]
    }
]

response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

**かなり良くなりました！**


## ローカルにない画像（URL の画像）を使う

ローカルに画像がない場合でも Claude に画像を渡せます。方法はいくつかありますが、基本レシピは同じです：

* リクエストライブラリなどで画像データを取得する
* バイナリを Base64 エンコードする
* bytes を UTF-8 で文字列にデコードする

ここでは `httpx` で URL から画像データを取得します。下の例の URL は、空にオーロラが出ている教会の画像です。


In [ ]:
import base64
import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Church_of_light.jpg/1599px-Church_of_light.jpg"
image_media_type = "image/jpeg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": image_data,
                    },
                },
                {
                    "type": "text",
                    "text": "Describe this image."
                }
            ],
        }
    ]


response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)




先ほどと同様に、URL から画像ブロックを生成するヘルパー関数も作れます。以下は、URL を受け取って次を行う軽量な実装例です：

* `httpx` で画像データを取得
* URL の拡張子から簡易的に MIME type を判定（最後の `.` の後ろを見るだけなので完全ではありません）
* 画像データを base64 エンコードして UTF-8 文字列にデコード
* Claude のプロンプトに入れられる形の画像ブロックを返す

例えば `get_image_dict_from_url("https://somewebsite.com/cat.png")` は、次の辞書を返します：

```python
{
    "type": "image",
    "source": {
        "type": "base64",
        "media_type": "image/png",
        "data": <actual image data>
    },
}
```


In [ ]:

def get_image_dict_from_url(image_url):
    # Send a GET request to the image URL and retrieve the content
    response = httpx.get(image_url)
    image_content = response.content

    # Determine the media type of the image based on the URL extension
    # This is not a foolproof approach, but it generally works
    image_extension = image_url.split(".")[-1].lower()
    if image_extension == "jpg" or image_extension == "jpeg":
        image_media_type = "image/jpeg"
    elif image_extension == "png":
        image_media_type = "image/png"
    elif image_extension == "gif":
        image_media_type = "image/gif"
    else:
        raise ValueError("Unsupported image format")

    # Encode the image content using base64
    image_data = base64.b64encode(image_content).decode("utf-8")

    # Create the dictionary in the proper image block shape:
    image_dict = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": image_media_type,
            "data": image_data,
        },
    }

    return image_dict

では試してみましょう。次の例では 2 つの画像 URL を使います：

* 消防車の PNG
* 緊急搬送ヘリコプターの JPG

画像はこちら：

![firetruck](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Rincon_fire_truck.png/1600px-Rincon_fire_truck.png)
Wikimedia Commons, CC-BY-SA
![helicopter](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Ornge_C-GYNP.jpg/1600px-Ornge_C-GYNP.jpg)
Wikimedia Commons, CC-BY-SA


この 2 枚を Claude に渡し、"What do these images have in common?"（共通点は？）と質問します：


In [ ]:
url1 = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Rincon_fire_truck.png/1600px-Rincon_fire_truck.png"
url2 = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/Ornge_C-GYNP.jpg/1600px-Ornge_C-GYNP.jpg"

messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Image 1:"},
                get_image_dict_from_url(url1),
                {"type": "text", "text": "Image 2:"},
                get_image_dict_from_url(url2),
                {"type": "text", "text": "What do these images have in common?"}
            ],
        }
    ]


response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

Claude は 2 枚とも緊急対応車両であることを正しく指摘できました。より重要なのは、URL から取得した画像を Claude に渡す方法も確認できたことです。


## Vision プロンプトのコツ


### 具体的に書く

テキストのみのプロンプトと同様、マルチモーダルでも具体的で詳細なプロンプトほど良い結果になりやすいです。例を見てみましょう。

次は友人グループの画像です。画像には 8 人いますが、うち 2 人は画像の端で途中までしか写っていません。


In [ ]:
from IPython.display import Image
Image(filename='./prompting_images/people.png') 

単純に Claude に「この画像には何人いますか？」と聞くと、おそらく 7 人という答えが返ってきます：


In [ ]:

messages=[
    {
        "role": "user",
        "content": [
            create_image_message("./prompting_images/people.png"),
            {"type": "text", "text": "How many people are in this image?"}
        ],
    }
]


response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

一方で、ステップバイステップで考えるように促し、数え上げが得意であること、画像の端で一部しか写っていない人も数えること、などを明示すると改善します：


In [ ]:
messages=[
    {
        "role": "user",
        "content": [
            create_image_message("./prompting_images/people.png"),
            {"type": "text", "text": "You have perfect vision and pay great attention to detail which makes you an expert at counting objects in images. How many people are in this picture? Some of the people may be partially obscured or cut off in the image or may only have an arm visible. Please count people even if you can only see a single body part. Before providing the answer in <answer> tags, think step by step in <thinking> tags and analyze every part of the image."}
        ],
    }
]


response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

### 例（Examples）を使う

例をプロンプトに入れると、テキストでも画像でも Claude の応答品質を改善できることがあります。

ここではスライドの画像を使い、Claude に「スライド内容の JSON 表現」を生成させることを目標にします。まず 1 枚目を見てください：


In [ ]:
from IPython.display import display
display(Image("./prompting_images/slide1.png", width=800))

目標は、スライドの背景色・タイトル・本文テキスト・画像の説明を含む JSON 形式の応答を生成してもらうことです。例えば上のスライドなら、JSON は次のようになります：

```json
{
    "background": "#F2E0BD",
    "title": "Haiku",
    "body": "Our most powerful model, delivering state-of-the-art performance on highly complex tasks and demonstrating fluency and human-like understanding",
    "image": "The image shows a simple line drawing of a human head in profile view, facing to the right. The head is depicted using thick black lines against a pale yellow background. Inside the outline of the head, there appears to be a white, spoked wheel or starburst pattern, suggesting a visualization of mental activity or thought processes. The overall style is minimalist and symbolic rather than realistic."
}
```


これは「例」を入れて Claude に望む出力形式を教えるのに向いたユースケースです。参考として、他にも 2 枚のスライド画像があります：


In [ ]:
display(Image("./prompting_images/slide2.png", width=800))

In [ ]:
display(Image("./prompting_images/slide3.png", width=800))

会話メッセージ形式を使って、前の入力と対応する出力の例を Claude に渡します：


In [ ]:

def generate_slide_json(image_path):

    slide1_response = """{
        "background": "#F2E0BD",
        "title": "Haiku",
        "body": "Our most powerful model, delivering state-of-the-art performance on highly complex tasks and demonstrating fluency and human-like understanding",
        "image": "The image shows a simple line drawing of a human head in profile view, facing to the right. The head is depicted using thick black lines against a pale yellow background. Inside the outline of the head, there appears to be a white, spoked wheel or starburst pattern, suggesting a visualization of mental activity or thought processes. The overall style is minimalist and symbolic rather than realistic."
    }"""

    messages = [
        {
            "role": "user",
            "content": [
                create_image_message("./prompting_images/slide1.png"),
                {"type": "text", "text": "Generate a JSON representation of this slide.  It should include the background color, title, body text, and image description"}
            ],
        },
        {
            "role": "assistant",
            "content": slide1_response
        },
        {
            "role": "user",
            "content": [
                create_image_message(image_path),
                {"type": "text", "text": "Generate a JSON representation of this slide.  It should include the background color, title, body text, and image description"}
            ],
        },
    ]

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=2048,
        messages=messages
    )
    print(response.content[0].text)


In [ ]:
display(Image("./prompting_images/slide2.png", width=800))
generate_slide_json("./prompting_images/slide2.png")

In [ ]:
display(Image("./prompting_images/slide3.png", width=800))
generate_slide_json("./prompting_images/slide3.png")

---

## 演習

この演習では、Claude を使って Anthropic の研究論文を文字起こしして要約します。`images` フォルダの中に `research_paper` フォルダがあり、論文のスクリーンショットが 5 枚入っています。すでに 5 枚の画像パスをリストで用意してあります：


In [ ]:
research_paper_pages = [
    "./images/research_paper/page1.png",
    "./images/research_paper/page2.png",
    "./images/research_paper/page3.png",
    "./images/research_paper/page4.png",
    "./images/research_paper/page5.png"
    ]

まず 1 枚目の画像を見てみましょう：


In [ ]:
Image(research_paper_pages[0])

### 課題

Claude を使って次を行ってください：

* 研究論文画像 5 枚それぞれのテキストを文字起こしする
* 各ページのテキストを結合して、1 つの大きな全文（transcription）にする
* その全文を Claude に渡し、非技術者向けに論文全体を要約してもらう

出力例は例えば次のようになります：

> This paper explores a new type of attack on large language models (LLMs) like ChatGPT, called "Many-shot Jailbreaking" (MSJ). As LLMs have recently gained the ability to process much longer inputs, this attack takes advantage of that by showing the AI hundreds of examples of harmful or undesirable behavior. The researchers found that this method becomes increasingly effective as more examples are given, following a predictable pattern.
>
> The study tested MSJ on several popular AI models and found it could make them produce harmful content they were originally designed to avoid. This includes things like violent or sexual content, deception, and discrimination. The researchers also discovered that larger AI models tend to be more susceptible to this type of attack, which is concerning as AI technology continues to advance.
>
> The paper also looked at potential ways to defend against MSJ attacks. They found that current methods of training AI to be safe and ethical (like supervised learning and reinforcement learning) can help somewhat, but don't fully solve the problem. The researchers suggest that new approaches may be needed to make AI models truly resistant to these kinds of attacks. They emphasize the importance of continued research in this area to ensure AI systems remain safe and reliable as they become more powerful and widely used.

より良い結果を得るために、5 枚まとめて 1 回で処理するよりも、各ページを別リクエストで要約していくのがおすすめです。


### 解答例


In [ ]:
import base64
import mimetypes

research_paper_pages = [
    "./images/research_paper/page1.png",
    "./images/research_paper/page2.png",
    "./images/research_paper/page3.png",
    "./images/research_paper/page4.png",
    "./images/research_paper/page5.png"
    ]

def create_image_message(image_path):
    # Open the image file in "read binary" mode
    with open(image_path, "rb") as image_file:
        # Read the contents of the image as a bytes object
        binary_data = image_file.read()
    
    # Encode the binary data using Base64 encoding
    base64_encoded_data = base64.b64encode(binary_data)
    
    # Decode base64_encoded_data from bytes to a string
    base64_string = base64_encoded_data.decode('utf-8')
    
    # Get the MIME type of the image based on its file extension
    mime_type, _ = mimetypes.guess_type(image_path)
    
    # Create the image block
    image_block = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": mime_type,
            "data": base64_string
        }
    }
    
    
    return image_block

def transcribe_single_page(page_url):
    messages = [
    {
        "role": "user",
        "content": [
            create_image_message(page_url),
            {"type": "text", "text": "transcribe the text from this page of a research paper as accurately as possible."}
        ]
    }
    ]

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=5000,
        messages=messages
    )
    return response.content[0].text

def summarize_paper(pages):
    complete_paper_text = ""
    for page in pages:
        print("transcribing page ", page)
        transribed_text = transcribe_single_page(page)
        print(transribed_text[:200])
        complete_paper_text += transribed_text
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=5000,
        messages=[
            {
                "role": "user",
                "content": f"This is the transcribed contents of a research paper <paper>{complete_paper_text}</paper>.  Please summarize this paper for a non-research audience in at least 3 paragraphs.  Make to sure explain any abbreviations or technical jargon, and use analogies when possible"
            }
        ]
    )
    print(response.content[0].text)



In [ ]:
summarize_paper(research_paper_pages)

***
